In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install texthero

In [ ]:
import re
from keras.preprocessing.sequence import pad_sequences


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Embedding,Activation, Dropout
from tensorflow.keras.layers import Conv1D,GlobalAveragePooling1D, MaxPooling1D, GlobalMaxPooling1D , LSTM , Bidirectional, SpatialDropout1D, BatchNormalization
from tensorflow.keras.optimizers import Adam,SGD
from keras import optimizers, callbacks 
from sklearn.metrics import log_loss
from keras.layers import Flatten
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from gensim.models import KeyedVectors
from keras.regularizers import l2
from keras.layers import Input
from keras.layers import Lambda
from keras.layers import concatenate,Reshape
import keras.backend as K
from keras import initializers
from keras.models import Model

import texthero as hero


In [ ]:
train_df = pd.read_csv('../input/quora-insincere-questions-classification/train.csv')
test_df = pd.read_csv('../input/quora-insincere-questions-classification/test.csv',  encoding='utf-8', engine='python')
train_df.head(5)
print("*"*50)
test_df.head(5)

In [ ]:
train_df['question_text'].str.split().map(lambda x : len(x)).hist(bins=64)

all question with max length is 45

In [ ]:
max_length = 45
embedding_glove = '../input/glove840b300dtxt/glove.840B.300d.txt'
embedding_para = '../input/paragram-300-sl999/paragram_300_sl999.txt'
WEIGHTS_FILE = './w0.h5'
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how does",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
" u ": " you ",
" ur ": " your ",
" n ": " and "}





In [ ]:
puncts = '\'!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2",
                 "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '”': '"', '“': '"', "£": "e",
                 '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta',
                 '∅': '', '³': '3', 'π': 'pi', '\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}
for p in puncts:
    punct_mapping[p] = ' %s ' % p

p = re.compile('(\[ math \]).+(\[ / math \])')
p_space = re.compile(r'[^\x20-\x7e]')
print(punct_mapping["∞"])

In [ ]:
all_text  = ' '.join(train_df['question_text'])

all_test  = ' '.join(test_df['question_text'])

all_text  = all_text.split()

all_test  = all_test.split()

frequence  = pd.Series(all_text).value_counts()
one_word = frequence[frequence.values == 1]
one_word[5:20]


token = Tokenizer()
token.fit_on_texts(all_text)
vocab_size  = len(token.word_index) + 1

token1 = Tokenizer()
token1.fit_on_texts(all_test)
vocab_size_test  = len(token1.word_index) + 1
    
print("vocab before cleaning" , vocab_size)  

print("vocab before testing" , vocab_size_test)  
    

accents = [ (u"é", u"e"), (u"ē", u"e"), (u"è", u"e"), (u"ê", u"e"), (u"à", u"a"),
                    (u"â", u"a"), (u"ô", u"o"), (u"ō", u"o"), (u"ü", u"u"), (u"ï", u"i"),
                    (u"ç", u"c"), (u"\xed", u"i")]
            

In [ ]:
def clean_question(x):
    if type(x) is str:
        x = x.lower() # transformer to lower 
        for p in punct_mapping:
            x = x.replace(p,punct_mapping[p])
        for key in contractions:
            value = contractions[key]
            x = x.replace(key, value)
            
        for i,j in accents :
             x = re.sub(i, j, x)
            
        
        x = re.sub(r'([a-zA-Z0-9+._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)', '', x) # regex to remove to emails
       
        x = re.sub(r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '', x)   #regex to remove URLs     
        x = re.sub( u"\s+", u" ", x ).strip() # remove multiple  espace and back line
        x = ' '.join([t for t in x.split() if t not in one_word])  # combining all the text excluding rare words.
        return x
    else:
        return x

train_df['question_text'] = train_df['question_text'].apply(lambda x: clean_question(x))
train_df['question_text'] = train_df['question_text'].tolist()

test_df['question_text'] = test_df['question_text'].apply(lambda x: clean_question(x))
test_df['question_text'] = test_df['question_text'].tolist()

In [ ]:
train_df[200:210]

preparing embedding matrix 

Lemmatization is the process of converting a word to its base form. The difference between stemming and lemmatization is, lemmatization considers the context and converts the word to its meaningful base form, whereas stemming just removes the last few characters, often leading to incorrect meanings and spelling errors.

In [ ]:
train_df['question_text'] = (train_df['question_text'].pipe(hero.remove_angle_brackets)
                    .pipe(hero.remove_brackets)
                    .pipe(hero.remove_curly_brackets)
                    .pipe(hero.remove_diacritics)
                    .pipe(hero.remove_round_brackets)
                    .pipe(hero.remove_square_brackets)
                    .pipe(hero.remove_punctuation)
                    .pipe(hero.remove_stopwords))
   

In [ ]:
train_df[200:210]

In [ ]:
test_df['question_text'] = (test_df['question_text'].pipe(hero.remove_angle_brackets)
                    .pipe(hero.remove_brackets)
                    .pipe(hero.remove_curly_brackets)
                    .pipe(hero.remove_diacritics)
                    .pipe(hero.remove_round_brackets)
                    .pipe(hero.remove_square_brackets)
                    .pipe(hero.remove_punctuation)
                    .pipe(hero.remove_stopwords))

In [ ]:
test_df[200:210]

In [ ]:
lemm = WordNetLemmatizer()

def word_lemma(text):
    words = nltk.word_tokenize(text)
    lemma = [lemm.lemmatize(word) for word in words]
    joined_text = " ".join(lemma)
    return joined_text

train_df['question_text'] = train_df['question_text'].apply(lambda x: word_lemma(x))

test_df['question_text'] = test_df['question_text'].apply(lambda x: word_lemma(x))

In [ ]:
train_df[200:210]

In [ ]:
test_df[200:210]

In [ ]:
def pre_embe(data ):
    token = Tokenizer()
    token.fit_on_texts(data)
    vocab_size  = len(token.word_index) + 1
    print(" vocabolury size :  " ,vocab_size)
    encoded_text = token.texts_to_sequences(train_df['question_text'])
    X = pad_sequences(encoded_text, maxlen=max_length, padding='post')
    print(" exemple of fisrt question with encoding index ",X[1])
    return X , vocab_size , token.word_index

X , vac , word_index= pre_embe(train_df['question_text'])
    
X_testing , vac_test , word_index_test = pre_embe(test_df['question_text'])

In [ ]:
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')

def load_embeddings(method):
    embeddings_index = dict()        

    if method=='glove':
        embeddings_index.update( dict(get_coefs(*o.split(" ")) for o in open(embedding_glove))) 
    if method=='paragram':
        embeddings_index.update( dict(get_coefs(*o.split(" ")) for o in open(embedding_para, encoding="utf8", errors='ignore'))) 
    
    return embeddings_index



def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')

def load_embeddings(method):
    if method=='glove':
        embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in tqdm(open(embedding_glove)))
    if method=='paragram':
        embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in tqdm(open(embedding_para))
    
    return embedding_index

In [ ]:
glove = load_embeddings('glove')

In [ ]:
glove.get('you').shape

In [ ]:
paragram = load_embeddings('paragram')

In [ ]:
paragram.get('you').shape

+------------+------------+
|   index    |  Embedding |
+------------+------------+
|     0      | [1.2, 3.1] |
|     1      | [0.1, 4.2] |
|     2      | [1.0, 3.1] |
|     3      | [0.3, 2.1] |
|     4      | [2.2, 1.4] |
|     5      | [0.7, 1.7] |
|     6      | [4.1, 2.0] |Embedding(7, 2, input_length=5)
+------------+------------+

In [ ]:
def convert_embidding(word_index,embeddings_index,vocabulaire_size) : 
    
    matrix_vector = np.zeros((vocabulaire_size, 300))

    for word, index in word_index.items():
        vector = embeddings_index.get(word)
        if vector is not None:
            matrix_vector[index] = vector
      
    return matrix_vector
    
    
embedding_matrix_glove    =  convert_embidding(word_index,glove,vac)  
embedding_matrix_paragram =  convert_embidding(word_index,paragram,vac)    
    
#Meta embidding    
    
embedding_matrix = np.mean([1.4 * embedding_matrix_glove, 
                            0.6 * embedding_matrix_paragram], 
                           axis = 0)


#matrix_vector = convert_embidding(word_index,glove,vac)    
    

In [ ]:
embedding_matrix_glove_test    =  convert_embidding(word_index_test,glove,vac_test)  
embedding_matrix_paragram_test =  convert_embidding(word_index_test,paragram,vac_test)    
    
#Meta embidding    
    
embedding_matrix_test = np.mean([1.4 * embedding_matrix_glove_test, 
                            0.6 * embedding_matrix_paragram_test], 
                           axis = 0)

In [ ]:
y = train_df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2, stratify = y)

In [ ]:
def model_create(matrix_vector , trainable=False , lr=0.001, seed = 42):
    
    np.random.seed(seed)
    
    x_input = Input(shape=(max_length,))
    
    
    emb = Embedding(vac, 300, input_length=max_length, weights = [matrix_vector], trainable = trainable)(x_input)
    
    x = SpatialDropout1D(0.2, seed=10000)(emb)
    
    
    rnn = Bidirectional(LSTM(128,activation = 'tanh',return_sequences=True))(x)
    
    maxlstm = GlobalMaxPooling1D()(rnn)
    
    maxlstm = BatchNormalization()(maxlstm)
    
    average = Lambda(lambda x : K.sum(x, axis=1))(rnn)

    average = BatchNormalization()(average)

    c = concatenate([maxlstm,average], axis=1)
    
    c = Reshape((2, -1))(c)
    
    x = Dense(100, activation='relu')(c)
    
    x = Dropout(0.2, seed=10000)(x)
    
    x = BatchNormalization()(x)
    
    x_output = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=x_input, outputs=x_output)
    
    model.compile(optimizer=optimizers.Adam(lr=lr), loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

The requirements to use the cuDNN implementation are:

activation == tanh
recurrent_activation == sigmoid
recurrent_dropout == 0
unroll is False
use_bias is True
Inputs, if use masking, are strictly right-padded.
Eager execution is enabled in the outermost context.

In [ ]:
model = model_create(embedding_matrix)

model.summary()

In [ ]:
def run_model(model , lr=1e-3, bs=2048):
    
    predictions_train  = pd.DataFrame()
    
    for seed in range(2):
        
        es = callbacks.EarlyStopping( patience=4 )
        mc = callbacks.ModelCheckpoint( filepath=WEIGHTS_FILE, monitor='val_loss', mode='min', save_best_only=True )
        
        history = model.fit(X_train, y_train, validation_data=(X_test, y_test), callbacks=[es, mc],
                            batch_size=bs, epochs=100, verbose=2 )
        
        model.load_weights(WEIGHTS_FILE)
        
        p = model.predict(X_testing, batch_size=bs)
        
        
        #p = model.predict(X_test_emb, batch_size = 2048)
        
        #predictions_test[str(seed)] = p.flatten()

        #p = model.predict(X_train_emb, batch_size = 2048)
        
        predictions_train[str(seed)] = p.flatten()

        #print ( 'BAGGING SCORE Test: ' , log_loss(y_test,  predictions_test.mean(axis=1), eps = 1e-7) )
        #print ( 'BAGGING SCORE Train: ', log_loss(y_train, predictions_train.mean(axis=1), eps = 1e-7) )
        
        print("test"+"*"*50)
        print(y_test[1:10])
        print("result_test"+"*"*50)
        print(p[1:10])
        
        plt.plot(history.history['val_loss'])
        plt.plot(history.history['loss'])
        plt.legend( ['test', 'train'] )
        
        print('****************************************')
        
        

In [ ]:
run_model(model , lr=1e-3,bs=2048)

In [ ]:
#model1 = model_create(embedding_matrix,trainable=True)

#print(model1.summary())

#model1.load_weights(WEIGHTS_FILE)

In [ ]:
#run_model(model1 , lr=1e-3,bs=2048)

In [ ]:
all_preds_sub = model.predict(X_testing, batch_size=2048)

In [ ]:
sub_df = pd.DataFrame()
sub_df['qid'] = df_sub.qid.values

for x in all_preds_sub:
    if x>0.34:
        x = 1
    else:
        x = 0
        
sub_df['prediction'] = all_preds_sub
sub_df.to_csv('submission.csv', index=False)


In [ ]:

#for lr in [1e-5, 1e-4, 1e-3]:
           
  #  run_model(model1, lr=lr,bs=2048)
            
   # print("**********************************************************************")
    
